# Intrusion Detection on the KDD Cup 99 Data Set: Documentation
*— Daniel Jones*

## Notebook Setup

Libraries for Python session (see `readme.md` for installation instructions):

In [ ]:
import matplotlib  # graphs and plotting for python
import numpy  # fast arrays for python (used by pandas)
import pandas  # provides dataframe's and similar structures for python
import seaborn  # provides pre-configured, pretty graphs for matplotlib

from sklearn import decomposition
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

Setup Jupyter with `rpy2` to allow embedding R, and `matplotlib` to allow inline plots.

In [ ]:
# For inline plots within the notebook
%matplotlib inline
# Allows code cells to be intrepreted as R (put %%R on the first line) [^1]
%load_ext rpy2.ipython
# Render R output as HTML
from functools import partial
from rpy2.ipython import html
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")
html.init_printing()

Libraries for R session (see `readme.md` for installation instructions):

In [ ]:
%%R
library(caret)
library(ggplot2)

In [ ]:
random_state = numpy.random.RandomState(0)

## Data Source

Each row in the data represents a single TCP connection, as described in the original task description [^2]:
> A connection is a sequence of TCP packets starting and ending at some well defined times, between which data flows to and from a source IP address to a target IP address under some well defined protocol.  Each connection is labeled as either normal, or as an attack, with exactly one specific attack type.



In [ ]:
columns=['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'connection_label']
connection_events = pandas.read_csv('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', names=columns)  # [^3]

In [ ]:
connection_events.head(10).transpose()

The `connection_label` column describes the source of each connection; either the name of the red-team which caused the event, or the string `normal.` which indicates normal network behaviour. 

The task is to create a model which can separate red-team behavour from normal network behaviour, so group the data into two labels: `normal` and `bad`.

In [ ]:
def generate_label(connection_label):
    return 'normal' if connection_label == 'normal.' else 'bad'

connection_events['connection_label'] = connection_events['connection_label'].apply(func=generate_label)

Next, separate out the labels from the data set. 

In [ ]:
connection_labels = connection_events.filter(['connection_label'], axis='columns')
connection_events = connection_events.drop(['connection_label'], axis='columns')

## Training and Testing Data

It is now necessary to split the data into training and testing sets. When doing this we should ask the following questions:

  1. Does the ratio of normal and bad connections need to be similar in the training and testing data? If so, we should use stratified testing.
    - Unless you include all types in the training set, the model won't necessarily be able to distinguish between the classes. Further, if we sample the incorrect ratio of classes in our training and testing sets, we have the potential to bias the model against rare classes.
  2. Should we consider k-fold validation?
    - What do we think? Yes: it helps to protect against overfitting of the data [^ref???] and allows us to better measure how our model will perform against unseen data.
    - This is quick to implement, but would require each of us to write our models in such a way that they are repeatable.
    
**TODO:** If our models perform well, we should consider doing a quick test at the end with new data (from the non-10percent data set). This was mentioned as a potential think to do in the assessment spec:
> ... for example, if you believe that you can predict new types of data, you could demonstrate this by leaving out some types of data and observing your performance.
    
Perform 10-fold cross-validation on the data, generating the `train_test_splits` variable mapping a training and testing set to each experiment. This process splits the data into ten bins (stratified to ensure a similar ratio of `normal` and `bad` connections in each group), then creates ten combinations of nine bins of training data to one bin of testing data. These are configured in such a way that each of the ten bins is used for testing at some point.

By doing this, we have ensured that every single data point has been left out of training and tested against. This should help protect against overfitting of data, and allows us to better measure how our model will perform against unseen data. Further, by doing this ten times, we have lowered the potential for "lucky dips" when randomly sampling training and test data sets (in contrast to the leave-one-out methodology).

When splitting data into training and testing sets, it is important to consider the ratio's of different classes. In our case, around $20\%$ of our connection events are classified as `normal` (with the resr being classified as `bad`). We should also consider whether having unbalanced class ratios could have an effect on either the model, or our performance metrics. In the case of the model, this should not have an affect <insert intution here based on how logistic regression works> and these sources (or there sources):
  - https://www.analyticbridge.datasciencecentral.com/forum/topics/handling-imbalanced-data-when-building-regression-models
  - http://www.win-vector.com/blog/2015/02/does-balancing-classes-improve-classifier-performance/
  - https://stats.stackexchange.com/questions/6067/does-an-unbalanced-sample-matter-when-doing-logistic-regression . This looks particularly good, and makes a good point: our class ratio's do not reflect real world data. (I would imagine) that the vast majority of traffic experienced by an companies internal network is going to be `normal`. Our data set is the opposite (20% normal). Does this have the potential to affect our performance?
    - In general, we should be aware that we are looking for *rare events*, and modify our analysis to represent this: https://statisticalhorizons.com/logistic-regression-for-rare-events
   - **potential suggestion** use penalized likelihood!
   - we should also talk about how the unrealistic data set has the potential to wrongly bias our models, and make them less applicable to real world scenarios.
  
If so, we should use stratified testing.
    - Unless you include all types in the training set, the model won't necessarily be able to distinguish between the classes. Further, if we sample the incorrect ratio of classes in our training and testing sets, we have the potential to bias the model against rare classes.
    
The code below performs 10-fold cross-validation, providing the array `train_test_splits`. It does this using the [model selection](http://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold) part of the scikit-learn library [^5].

In [ ]:
k_fold_splitter = model_selection.StratifiedKFold(n_splits=10,  random_state=random_state)
train_test_splits = k_fold_splitter.split(
    connection_events,  # data to be split
    connection_labels,  # target/class to split by
)

# Force evaluation of the train_test_splits generator into a list. This needs to be
# done before it's sent to R.
train_test_splits = [
    [training_indexes, testing_indexes]
    for training_indexes, testing_indexes in train_test_splits
]

# Here train_test_splits is a list, where each item represents a single 90/10 split of 
# training and testing data respectively. The values contained are 0-based indexes
# of the rows in each part of the split, i.e:
#
#   train_test_splits = [
#      (indexes_of_training_samples, indexes_of_test_samples),  # first split
#      (indexes_of_training_samples, indexes_of_test_samples),  # second split
#      ...
#      (indexes_of_training_samples, indexes_of_test_samples),  # kth split
#  ]
#
# These indexes can then be used to fetch the data samples and their labels,
# ready for training and testing.

The cell below imports the training and testing sets into the R session, ready for analysis and modelling.

In [ ]:
%Rpush train_test_splits connection_events connection_labels

Below are helper functions to extract the training and test data from a data frame using an item in the `train_test_splits` array:

In [ ]:
%%R

get_training_rows <- function(dataframe, train_test_split) {
    # Training indexes are the first item in a train_test_split
    indexes <- train_test_split[[1]]
    
    # Python indices start at 0, whilst R indices start at 1.
    # Correct for this by incrementing each index by 1:
    indexes <- indexes + 1
    
    dataframe[indexes,]
}

get_testing_rows <- function(dataframe, train_test_split) {
    # Testing indexes are the second item in a train_test_split
    indexes <- train_test_split[[2]]
    
    # Python indices start at 0, whilst R indices start at 1.
    # Correct for this by incrementing each index by 1:
    indexes <- indexes + 1  
    
    dataframe[indexes,]
}

The following shows how to extract the first set of training and testing data using the functions above:

In [ ]:
%%R

# Use the indexes to get out the training data
training_data <- get_training_rows(connection_events, train_test_splits[[1]])
training_labels <- get_training_rows(connection_labels, train_test_splits[[1]])

# Use the indexes to get out the testing data
testing_data <- get_testing_rows(connection_events, train_test_splits[[1]])
testing_labels <- get_testing_rows(connection_labels, train_test_splits[[1]])

## Data Model

Here's a quick demo of using the data within the R session, and creating a model based upon it. 

In [ ]:
%R head(testing_data)

In [ ]:
%%R
qplot(x=src_bytes, y=dst_bytes, data=testing_data, geom='point')

In practice, we would want to do the folllowing for each train/test split $i$ :
  1. Train the model based on the $i$-th set of training data and labels.
  2. Apply this trained model on the $i$-th set of testing data.
  3. Save the set of predicted labels.
  
Then return a vector of each of these sets of predicted labels.

For this example, come up with a silly set of predicted labels for each of our 10 training/testing splits. This is the format of predicted labels expected by the performance analsysi in the next section.

In [ ]:
%%R
predicted_label_sets <- list(
    sample(c('normal', 'bad'), nrow(train_test_splits[[1]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[2]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[3]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[4]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[5]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[6]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[7]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[8]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[9]][[2]]), replace=TRUE, prob=c(0.5, 0.5)),
    sample(c('normal', 'bad'), nrow(train_test_splits[[10]][[2]]), replace=TRUE, prob=c(0.5, 0.5))
)

### Logistic Regression
  - Info on Binary Logistic Regression : https://www.statisticssolutions.com/binary-logistic-regression/
  - Logit model in R: https://onlinecourses.science.psu.edu/stat504/node/225/
  - Practical:
    - https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python
    - https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic_l1_l2_sparsity.html#sphx-glr-auto-examples-linear-model-plot-logistic-l1-l2-sparsity-py
    - https://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html#sphx-glr-auto-examples-linear-model-plot-iris-logistic-py
    
This plot might be useful for explanation: https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic.html#sphx-glr-auto-examples-linear-model-plot-logistic-py
  
 
### Data Preparation
Do I need to prepare the data in any way?
  - **TODO** Transform categorical columns into numeric columns?
  - **TODO** Normalization maybe?
  - **TODO** More?
  

##### Categorical Columns

Choose to do this using one-hot encoding as this avoids assigning implicit meaning or relationships between the categories. For example, if we naively assigned the categories in `protocol_type` the numbers `1,2,3`, we may end up implying to the model that $\text{tcp} \gt \text{icmp}$.

In [ ]:
connection_events.select_dtypes(include=[object]).describe()

So need to transform the `protocol_type`, `service` and `flag` columns. Consider their values:

In [ ]:
set(connection_events.protocol_type), set(connection_events.service), set(connection_events.flag)

In [ ]:
connection_events = pandas.get_dummies(connection_events, columns=["protocol_type", "service", "flag"])

The one-hot-encoding performed by, `pandas.get_dummies`, transforms a single feature that can take $n$ values, into $n$ features that can take $2$ values. This can be seen below:

In [ ]:
connection_events.filter(regex='(flag|protocol_type|service).*', axis='columns').head()

The following code runs scikit-learn's [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) [^5] implementation on each of the train/test splits, saving it's predictions for analysis.

In [ ]:
testing_predictions = []

for training_indices, testing_indices in train_test_splits:                                      
    training_data = connection_events.iloc[training_indices]
    training_labels = connection_labels.iloc[training_indices]

    logistic_regression = linear_model.LogisticRegression()
    logistic_regression.fit(training_data, training_labels)
    
    testing_data = connection_events.iloc[testing_indices]
    predicted_labels = logistic_regression.predict(testing_data)
    
    testing_predictions.append(predicted_labels)

**TODO** Describe scikit-learns logisitic regression functions, and how the differ from that which is described in a textbook:
  1. They use L1 penalisation by default, what does that mean?
  2. They try to find an optimal probability threshold automatically, how do they do it and why?
    - Plot the probabilities with the computed threshold.
  3. Did they make any other interesting choices?

     
     
 ### Feature Selection    
Feature selection:
  - scikit-learn: https://scikit-learn.org/stable/modules/feature_selection.html
  - https://machinelearningmastery.com/feature-selection-in-python-with-scikit-learn/
  - Could do PCA/SVD and look at the first few principle components:
    - their eigenvectors will each weight the original features by important.
    - Using this we could either:
      - do logistical regresion on the principle components,
      - or, identify the most important features from the PCA.
      - talked about here: https://stackoverflow.com/questions/34052115/how-to-find-the-importance-of-the-features-for-a-logistic-regression-model/34052747
  - "Selecting Features for Intrusion Detection: A Feature Relevance Analysis on KDD 99 Intrusion Detection Datasets"
     -  https://pdfs.semanticscholar.org/1d6e/a73b6e08ed9913d3aad924f7d7ced4477589.pdf
     -  uses *binary discriminant analysis* to pick out features
     
     
#### Why is it interesting?

In real-world scenarios we will often want our classification system to work in real-time, identifying threats as soon as possible after they happen. Reducing the number of features has the potential to speed up the execution time of our classification algorithms, allowing new threats to be found and acted upon promptly.

From "Characterization and classification of malicious Web traffic" [^8]:
>  In addition to learning about characteristics of malicious activities, reducing the number of features by removing the irrelevant and noisy features speeds up the machine learning algorithms and hopefully improves their performance (Liu and Yu, 2005) [^9].

  

#### Using PCA to select rank feature relevance
  - https://scikit-learn.org/stable/modules/decomposition.html#pca
  - https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
pca = decomposition.PCA(
    svd_solver='full',  # force sci-kit learn to compute the exact PCA rather than an estimation
    random_state=random_state,
)

pca.fit(connection_events)

In [ ]:
pc0 = pca.components_[0]
sorted(zip(connection_events.columns, pc0), key=lambda pair: abs(pair[1]))[-5:]

In [ ]:
pc1 = pca.components_[1]
sorted(zip(connection_events.columns, pc1), key=lambda pair: abs(pair[1]))[-5:]

In [ ]:
pc2 = pca.components_[2]
sorted(zip(connection_events.columns, pc2), key=lambda pair: abs(pair[1]))[-5:]

So the above looks at the three most important principle components, and looks at which of the original features they each value. They are each > 99% comprised of a single feature. In order of importance: src_bytes, dst_bytes, duration.

We should do a plot to find out how much each of the PC's contributes, then select the highly weighted features for each of the top-ranked PCs. Then:

  1. Re-run the regression on only those features
  2. Re-run the regression on the first few PC's
  3. Do the reverse and find the least important features, then remove them from the data set and run it again.

### Logistic Regression with Additional Covariates


### Logistic Regression with Penalized Likelihood


### Another approach

## Model Performance and Analysis


Import the predictions from the R session into Python:

In [ ]:
%Rpull predicted_label_sets

In [ ]:
# rpy2 returns an r-type list of character vectors. Convert each set of predictions into a numpy 
# array for processing with numpy/pandas/sklearn etc.
predicted_label_sets = [numpy.array(predicted_labels) for predicted_labels in predicted_label_sets]

## Do the above if we want to evaluate the R model, but we don't, evaluate the predictions from the Python model instead:
predicted_label_sets = testing_predictions

The performance analysis could be done in R or Python. Here is an example in Python [^4]:

In [ ]:
true_label_sets = [connection_labels.iloc[testing_indexes] for training_indexes, testing_indexes in train_test_splits]

In [ ]:
confusion_matrixes = [
    metrics.confusion_matrix(true_labels, predicted_labels)
    for true_labels, predicted_labels
    in zip(true_label_sets, predicted_label_sets)
]

In [ ]:
summary_confusion_matrix = sum(confusion_matrixes)
summary_confusion_matrix = pandas.DataFrame(
    data=summary_confusion_matrix, 
    index=['True Normal', 'True Bad'], 
    columns=['Predicted Normal', 'Predicted Bad'],
)

summary_confusion_figure, summary_confusion_axes = matplotlib.pyplot.subplots()
summary_confusion_axes.set_title(
    'Confusion matrix showing the predicted vs. true \n'
    'class of "normal" and "bad" network connections.'
)
seaborn.heatmap(
    summary_confusion_matrix,
    annot=True,
    fmt="d",
    cmap=seaborn.color_palette("Blues"),
    vmin=0,
    ax=summary_confusion_axes,
)

In [ ]:
def sensitivity(confusion_matrix):
    true_positives = confusion_matrix['Predicted Normal']['True Normal']
    false_negatives = confusion_matrix['Predicted Bad']['True Normal']
    return true_positives/(true_positives+false_negatives)

print('Sensitivity: {:.2f}%'.format(
    sensitivity(summary_confusion_matrix)*100
))

In [ ]:
def specificity(confusion_matrix):
    false_positives = confusion_matrix['Predicted Normal']['True Bad']
    true_negatives = confusion_matrix['Predicted Bad']['True Bad']
    return true_negatives/(true_negatives+false_positives)

print('Specificity: {:.2f}%'.format(
    specificity(summary_confusion_matrix)*100
))

Unsuprisingly, modelling the traffic as being uniformly distributed with a 50/50 split did not work particularly well. In fact, it gave sensitivity and specificity measures of approximately $50\%$.

Potential Approaches:
  -  Info on PRESS(predicted residual error sum of squares): https://en.wikipedia.org/wiki/PRESS_statistic
  - ROC Curve 
    - <https://en.wikipedia.org/wiki/Receiver_operating_characteristic>
    - Practical Statistics for Data Science, 1st ed., by Peter Bruce and Andrew Bruce (O’Reilly Media, 2017).


### Notes on real-world context

The dataset contains $20\%$ normal traffic and $80\%$ malicious. This is consistent with recent real-world analysis on public-facing web services performed by Incapsula [^6]. Our use case, however, is intrusion detection within internal networks [^2], where malicious traffic should be a rarity.

This means that  performance metric focused on the KDD Cup 99 data set may not be applicable to models used for real-world scenarios. In particular, an overly sensitive model would be favoured in the KDD Cup 99 data set, since its error would be smaller in ratio compared to a real world data set, which may have less than $5\%$  malicious traffic internally.

## Introduction

The KDD Cup 99 data set [^3], being one of the few publically available data sets of this size and nature, has receieved a large amount of study and analysis within academic community. This has provided a pool of prior work to learn from. 

In particular, Katos 2007 [^7] evaluated three key approaches to detecting normal and malicious connections within the KDD 99 data set  - cluster, discriminant and logit analysis.

## References

[^1]: rpy2, https://rpy2.bitbucket.io/.

[^2]: KDD-CUP-99 Task Description, http://kdd.ics.uci.edu/databases/kddcup99/task.html.

[^3]: Hettich, S. and Bay, S. D. (1999). The UCI KDD Archive [http://kdd.ics.uci.edu]. Irvine, CA: University of California, Department of Information and Computer Science.

[^4]: Data Science Toolbox: Assignment 1, https://github.com/dj311/data-science-toolbox-1.

[^5]: Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

[^6]: Incapsula: Annual Bot Traffic Report, https://www.incapsula.com/about/press-releases/incapsula-finds-malicious-bots-account-for-approximately-30-percent-of-internet-traffic/

[^7]: Katos, V., 2007. Network intrusion detection: Evaluating cluster, discriminant, and logit analysis. Information Sciences, 177(15), pp.3060-3073. https://doi.org/10.1016/j.ins.2007.02.034

[^8]: Goseva-Popstojanova, Katerina, et al. "Characterization and classification of malicious Web traffic." computers & security 42 (2014): 92-115. https://doi.org/10.1016/j.cose.2014.01.006

[^9]: Liu, Huan, and Lei Yu. "Toward integrating feature selection algorithms for classification and clustering." IEEE Transactions on knowledge and data engineering 17.4 (2005): 491-502. https://doi.org/10.1109/TKDE.2005.66